In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import json
import ast
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def complectation_proccesing(x):
    '''Достает список доступных опций модели'''
    if pd.isnull(x):
        return x
    else:
        try:
            dict_x = json.loads(x)
            return list(dict_x.values())[2]
        except:
            try:
                dict_x = ast.literal_eval(x)
                return list(dict_x.values())[2]
            except:
                return np.nan

          
            
def get_dicts(x):
    '''Преобразует строку в словарь'''
    if pd.isnull(x):
        return {'temp': 'temp'}
    else:
        try:
            return json.loads(x)
        except:
            return ast.literal_eval(x)

        
        
def pct_missing(df, column):
    '''Фильтр признаков'''
    missing = df[column].isna().mean()*100
    if missing > 40: # Сила фильтрации
        return column
        
        
        
def fill_missing(df, col_x, col_y):
    '''
    Функция для заполнения пропусков.
    
    Группирует столбец y по количеству значений. 
    Затем заменяет пропуски  в столбце x на максимальное 
    значение из сгруппированного датафрейма.
    '''
    y_counts = df.groupby([col_y])[col_x].value_counts().unstack().T
    values_lst = list(y_counts.columns)
    filled_col = df[[col_x, col_y]].apply(lambda x: y_counts[x[1]].idxmax() if 
                                          pd.isna(x[0]) and x[1] in values_lst else x[0], axis=1)
    return filled_col

In [ ]:
data = pd.read_csv('temp_data.csv')

In [ ]:
'''Преобразуем и сохраняем значения в переменные'''
complect = data['complectation'].apply(complectation_proccesing)
equip = data['equipment'].apply(get_dicts)
gen = data['super_gen'].apply(get_dicts)

In [ ]:
'''Создаем датафреймы на основе переменных со словарями'''
equip_df = pd.DataFrame(list(equip))
gen_df = pd.DataFrame(list(gen))

In [ ]:
'''
В equip_df получилось слишком много признаков.
Чистим от неинформативных.
'''
equip_drop_lst = ['PW1','8T6','PL2','9S6','9P5','RAB','PN0','PB0','6XE','PX2','3L5','6NQ','PNQ','9R1','KA2','4D3','235','242',
 '246','249','270','274','351','364','367','431','448','475','500','501','513','551','670','728','772','776','807','810','824',
 '871','873','881','882','883','889','927','989','U09','51U','H06','U22','DZ9','P17','B63','RVR','14U','R66','P29','03B','P31',
 '16U','P44','R01','B16','17U','L5C','WVA','S7N','E0P','2C5','1N1','7AA','9AB','UI4','6XD','3B3','7B2','9S3','4I2','3NZ','4UC',
 '4UE','4A3','4R2','4R3','6R1','9T1','3Q6','temp']

equip_df.drop(equip_drop_lst, axis=1, inplace=True)

In [ ]:
'''Заполняем часть пропусков на основе списков с опциями из complect'''
equip_df_col = list(equip_df.columns)
equip_df['complect'] = complect
for col in tqdm(equip_df_col):
    equip_df[col] = equip_df[[col, 'complect']].apply(lambda x: True if col in str(x[1]) else x[0], axis=1)

In [ ]:
'''Чистим признаки на основе данных о complect'''
drop_cols = []
for col in equip_df_col:
    drop_cols.append(pct_missing(equip_df[equip_df['complect'].notnull()], col))
drop_cols = list(filter(None, drop_cols))
equip_df.drop(drop_cols+['complect'], axis=1, inplace=True)

In [ ]:
'''
Заполняем часть пропусков в зависимости от модели.
Остальные заполняем False
'''
equip_df_col = list(equip_df.columns)
equip_df['model'] = data['model_name']

for col in tqdm(equip_df_col):
    equip_df[col] = fill_missing(equip_df, col, 'model')

equip_df.drop(['model'], axis=1, inplace=True)
equip_df = equip_df.fillna(False)

In [ ]:
'''Фильтруем датафрейм по интересующим нас признакам'''
gen_df_cols = ['acceleration', 'clearance_min', 'fuel_rate']
gen_df = gen_df[gen_df_cols]

In [ ]:
'''Заполняем часть пропусков в зависимости от модели'''
gen_df['model'] = data['model_name']
for col in tqdm(gen_df_cols):
    gen_df[col] = fill_missing(gen_df, col, 'model')

gen_df.drop(['model'], axis=1, inplace=True)

In [ ]:
'''Заполняем часть пропусков в зависимости от мощности двигателя'''
gen_df['power'] = data['engine_power']
for col in tqdm(gen_df_cols):
    gen_df[col] = fill_missing(gen_df, col, 'power')

gen_df.drop(['power'], axis=1, inplace=True)

In [ ]:
'''Заполняем часть пропусков в зависимости от объема двигателя'''
gen_df['volume'] = data['engine_volume']
for col in tqdm(gen_df_cols):
    gen_df[col] = fill_missing(gen_df, col, 'volume')

gen_df.drop(['volume'], axis=1, inplace=True)

In [ ]:
'''Заполняем оставшиеся пропуски'''
gen_df['acceleration'] = gen_df['acceleration'].fillna(gen_df['acceleration'].min())
gen_df['fuel_rate'] = gen_df['fuel_rate'].fillna(gen_df['fuel_rate'].min())
gen_df['clearance_min'] = gen_df['clearance_min'].fillna(round(gen_df['clearance_min'].mean()))

In [ ]:
'''Объединяем датасеты и сохраняем в файл'''
final_df = equip_df.join(gen_df)
final_df.to_csv('dict_data.csv', index=False)